<a href="https://colab.research.google.com/github/sarim711/BART_Kmeans_TextSummarisation/blob/main/Text_Summarisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generating medical text

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from tqdm import tqdm
tqdm.pandas()


In [ ]:
df = pd.read_csv("mg_small.csv",usecols = ["text"])

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 10)  # Adjust as needed to avoid truncation
df.head(2)

,text
0,"Nursing progress Note\n\n\n#1 Resp: rec'd pt on HIFI map 13, delta p 24. CBG at 1530\n7.22/58/43/25/-4, delta p increased to 26. FIO2 36-43% for\nmost of this shift. Pt increasingly labile/aggitated toward\nend of shift w/ frequent drifts to 80's requiring increased\nO2. Ett sxt for mod amt w/ cares, mouth sxt q4 for mod white\nsecretions. Cont. on combivent/Vit A. Cont. to monitor.\n#3 F/N: TF 130cc/kg of PND12 w/IL infusing via central PICC\nalong with MSO4 drip. Abd round, firm, ag 21 (above\npenrose), bowel sounds not audible, no stool. Pt's u/o\n6.0cc/kg/hr. D stick 62. Repogle tube to low intermittant\nsxt, minimal output. Penrose drain sutured in place,\ndraining sm amt serous fluid. Stable. Cont. to monitor. #5\nParenting: Mom/Dad in today. Mom assisting w/ cares, updated\nat bedside by [** 7**]. Asking appropriate questions about plan\nof care. Cont. to support/update. #6 DEV: temps stable on\nopen warmer. [**Known lastname 1348**] was alert/active this am, opening eyes\nduring care time. [**Known lastname 1348**] increasingly aggitated as shift\nprogressed, less tolerant of intervention. MAE. Fontanels\nsoft/flat. Cont. to support dev. needs. #7\nHyperbilirubinemia: NPO, no stool. Photo d/c'd [**2115-7-30**]. Draw\nbili in am. Cont. to monitor. #9 CV: hr 160-170's,\npink/ruddy-well perfused. No murmur audible this am. BP\n81/31 (50). Cont. to monitor. #10 ID: cont. on IV\n[**Month/Day/Year 1358**]/gent/vanco/zosyn/fluconazole. PICC line culture drawn\nthis evening. Other PICC cultures pending. Cont. to monitor.\n#11 Alt. pain: Pt maintained w/ Fentanyl drip and prn\nboluses today. With increasing aggitation, MSO4 boluses\ninitiated and morphine drip started at 1900. Fentanyl now\nd/c'd. Efforts made to decrease interventions/stimulation.\nCont. to monitor pt's comfort. See flowsheet for further\ndetails.\n\n\n\n"
1,"(Continued)\ning. CPP improved w/initiation of Neo gtt, currently @ 1mcg/kg/min. HR increased to 110s w/decrease in CVP, LR bolus given-cont to monitor & assess fluid status. pt currently febrile, tylenol given x 1, cont to monitor.\n\nPLAN: plan for Head CT this AM. cont q1hr neuro checks. cont to monitor neuro status, ICP, CPP, EVD & subdural drains & drainage, pupillary response, UOP, hemodynamics-assess need for fluid. cont to titrate Neo gtt for MAP >65. cont to replete lytes. cont to support pt & family.\n"


In [ ]:
def clean(text):

     '''removes anything other than standard punctuation
        and letters. Sentences should be separated by a period and a single space.
        using re for this.'''
    #Remove non-standard characters
    # text = re.sub(r'[^a-zA-Z\.\s]', '', text)

    #After updating the code to include numbers
    text = re.sub(r'[^a-zA-Z0-9\.\s]', '', text)

    #Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)

    #Ensure proper sentence separation (period followed by a single space)
    text = re.sub(r'\.(?=[^\s])', '. ', text)

    #Strip leading/trailing spaces
    text = text.strip()

    return text

print(clean(df.text.iloc[0]))
df.cleaned = df.text.apply(clean)

Nursing progress Note 1 Resp recd pt on HIFI map 13 delta p 24. CBG at 1530 7. 225843254 delta p increased to 26. FIO2 3643 for most of this shift. Pt increasingly labileaggitated toward end of shift w frequent drifts to 80s requiring increased O2. Ett sxt for mod amt w cares mouth sxt q4 for mod white secretions. Cont. on combiventVit A. Cont. to monitor. 3 FN TF 130cckg of PND12 wIL infusing via central PICC along with MSO4 drip. Abd round firm ag 21 above penrose bowel sounds not audible no stool. Pts uo 6. 0cckghr. D stick 62. Repogle tube to low intermittant sxt minimal output. Penrose drain sutured in place draining sm amt serous fluid. Stable. Cont. to monitor. 5 Parenting MomDad in today. Mom assisting w cares updated at bedside by 7. Asking appropriate questions about plan of care. Cont. to supportupdate. 6 DEV temps stable on open warmer. Known lastname 1348 was alertactive this am opening eyes during care time. Known lastname 1348 increasingly aggitated as shift progressed l

<ipython-input-4-448899ffb7b9>:25: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.cleaned = df.text.apply(clean)


In [ ]:
from collections import Counter
def calculate_word_frequency(document):
    '''return a dictionary that contains the
       frequencies of each word in the document'''
    #tokenize document into words
    words = re.findall(r'\b\w+\b', document.lower())

    # Calculate the frequency of each word
    word_count = Counter(words)

    # Normalize the frequency by the total number of words
    total_words = sum(word_count.values())
    word_freq = {word: count / total_words for word, count in word_count.items()}

    return word_freq

word_freq = calculate_word_frequency(df.cleaned.iloc[0])
word_freq

{'nursing': 0.0035587188612099642,
 'progress': 0.0035587188612099642,
 'note': 0.0035587188612099642,
 '1': 0.0035587188612099642,
 'resp': 0.0035587188612099642,
 'recd': 0.0035587188612099642,
 'pt': 0.010676156583629894,
 'on': 0.014234875444839857,
 'hifi': 0.0035587188612099642,
 'map': 0.0035587188612099642,
 '13': 0.0035587188612099642,
 'delta': 0.0071174377224199285,
 'p': 0.0071174377224199285,
 '24': 0.0035587188612099642,
 'cbg': 0.0035587188612099642,
 'at': 0.010676156583629894,
 '1530': 0.0035587188612099642,
 '7': 0.010676156583629894,
 '225843254': 0.0035587188612099642,
 'increased': 0.0071174377224199285,
 'to': 0.042704626334519574,
 '26': 0.0035587188612099642,
 'fio2': 0.0035587188612099642,
 '3643': 0.0035587188612099642,
 'for': 0.014234875444839857,
 'most': 0.0035587188612099642,
 'of': 0.017793594306049824,
 'this': 0.014234875444839857,
 'shift': 0.010676156583629894,
 'increasingly': 0.0071174377224199285,
 'labileaggitated': 0.0035587188612099642,
 'towar

In [ ]:
def calculate_sentence_scores(document, word_freq):
    '''Calculating the representativeness score of each sentence
       by summing up the frequency of the words in each sentence,
       then dividing by sentence length'''
    # Split the document into sentences
    sentences = re.split(r'(?<!\w\.\w\.)(?<![A-Z][a-z]\.)(?<=\.\s|\?\s|\!\s)', document)

    # Calculate scores for each sentence
    scores = []
    for sentence in sentences:
        words = re.findall(r'\b\w+\b', sentence.lower())
        sentence_score = sum(word_freq.get(word, 0) for word in words)
        if len(words) >5 :
            sentence_score /= len(words)  # Normalize by sentence length
        scores.append(sentence_score)

    return scores

sentence_scores = calculate_sentence_scores(df.cleaned.iloc[0], word_freq)
sentence_scores[:3]

[0.005338078291814948, 0.028469750889679714, 0.011862396204033216]

In [ ]:
def get_top_k_sentences(document, sentence_scores, k):
  '''Returning a list containing the top k most representative sentences'''
    # Split the document into sentences
    sentences = re.split(r'(?<!\w\.\w\.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', document)

    # Pair each sentence with its score and sort by score (descending order)
    ranked_sentences = sorted(zip(sentences, sentence_scores), key=lambda x: x[1], reverse=True)

    # Select the top-k sentences
    top_k_sentences = [sentence for sentence, score in ranked_sentences[:k]]

    return top_k_sentences
top_k_sentences = get_top_k_sentences(df.cleaned.iloc[0], sentence_scores,3)
top_k_sentences

['to monitor pts comfort.', 'to monitor.', 'to monitor.']

In [ ]:

def create_summary(document, top_k):
    cleaned = clean(document)
    word_freq = calculate_word_frequency(cleaned)
    sentence_scores = calculate_sentence_scores(cleaned, word_freq)
    top_k_sentences = get_top_k_sentences(document, sentence_scores, top_k)
    summary = ". ".join(top_k_sentences)
    return summary

df["statistical_summary"] = df.text.progress_apply(lambda text: create_summary(text, 3))

100%|██████████| 20000/20000 [00:12<00:00, 1546.26it/s]


In [ ]:
# Adjust pandas settings to display full lines
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 10)  # Adjust as needed to avoid truncation

df['statistical_summary']

,statistical_summary
0,"\n\n\n. #3 F/N: TF 130cc/kg of PND12 w/IL infusing via central PICC\nalong with MSO4 drip.. Mom assisting w/ cares, updated\nat bedside by [** 7**]."
1,cont to support pt & family.. cont to replete lytes.. cont q1hr neuro checks.
2,wife and daughter in room.. pt very needy... pt also received doses of ativan 2 mg iv and haldol 5 mg iv times 2 .
3,"status, continue\ncaffeine.. No bradys, frequent\ndrifts in sats to 80s.. O: Pt. orally intubated, 25/5, rate of 21."
4,Sinus bradycardia\n- premature ventricular contractions\nPossible old anterior infarct\nLateral/inferior ST-T changes are nonspecific\n\n
...,...
19995,"Normal sinus rhythm, rate 97\nLeft atrial enlargement\nBorderline low voltage in frontal leads\nPrior Inferior myocardial infarct\nModest Nonspecific lateral T waves abnormalities\nSince last ECG, no significant change\nAbnormal ECG\n\n"
19996,Condition slightly improved this pm.. Cxr was done this pm which only showed elevated diaphrams due to distended abdomen.. Resp.
19997,"\nWt 1315, no change.. CXR c/w CLD.. BP 62/27, 41."
19998,New ETT #7.0 taped 22cm@lip.. Moderate amount of blood tinged secretions noted after re-intubation.. Pt noted to have increase WOB and stridor and was fiberoptically reintubated by Anesthesia.


## K-means

We now graduate to using neural networks to improve our summarizations. The way this is done is by the following procedure:

1) Embed each sentence using bio-medical BERT (use whichever model you prefer, I enjoy PubMedBERT). Unfortunately, we cannot use the CLS token to create a general sentence embedding because that requires task-specific downstream training for a labeled dataset. Therefore, we take the average over the tokens in the sentence to create an embedding.

2) Cluster the sentence embeddings using [K-Means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).

3) For each cluster, choose the sentence that is closest to that cluster's centroid. Order the sentences according to their in-text appearance.

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM,AutoModel
import torch

# Check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load the PubMedBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract")
model = AutoModel.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract").to(device)

def embed_using_bert(bert_model, tokenizer, document):
    sentences = document.split(". ")
    sentence_embeddings = []

    for sentence_index, sentence in enumerate(sentences):
        # Tokenize the sentence and move input tensors to GPU if available
        tokens = tokenizer(sentence, return_tensors="pt", truncation=True, max_length=512).to(device)

        # Extract embeddings for tokens
        with torch.no_grad():
            outputs = bert_model(**tokens)

        # Get the token embeddings (hidden states of the last layer)
        token_embeddings = outputs.last_hidden_state.squeeze(0)

        # Compute the mean embedding for the sentence
        sentence_embedding = token_embeddings.mean(dim=0).cpu().numpy()  # Move back to CPU
        sentence_embeddings.append(sentence_embedding)

    return sentences, np.array(sentence_embeddings)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/225k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
from sklearn.cluster import KMeans

def cluster_sentence_embeddings(sentence_embeddings, sentences, k_clusters):
    # Remove duplicate embeddings
    unique_embeddings, unique_indices = np.unique(sentence_embeddings, axis=0, return_index=True)
    unique_sentences = [sentences[i] for i in unique_indices]

    # Adjust k to the number of unique embeddings
    k_clusters = min(k_clusters, len(unique_embeddings))

    if k_clusters == 0:  # No sentences to cluster
        return ""

    # Perform K-Means clustering
    kmeans = KMeans(n_clusters=k_clusters, random_state=42).fit(unique_embeddings)
    cluster_attributions = kmeans.labels_
    cluster_centers = kmeans.cluster_centers_

    # Compute the sentence closest to each cluster center
    representative_sentences = []
    for cluster_idx in range(k_clusters):
        cluster_indices = np.where(cluster_attributions == cluster_idx)[0]
        if len(cluster_indices) == 0:
            continue  # Skip empty clusters
        cluster_embeddings = unique_embeddings[cluster_indices]
        cluster_centroid = cluster_centers[cluster_idx]

        # Find the sentence closest to the centroid
        closest_idx = cluster_indices[np.argmin(np.linalg.norm(cluster_embeddings - cluster_centroid, axis=1))]
        representative_sentences.append((unique_indices[closest_idx], unique_sentences[closest_idx]))

    # Order sentences by their appearance in the original document
    representative_sentences.sort(key=lambda x: x[0])
    k_sentences_in_order = [sentence for _, sentence in representative_sentences]

    return ". ".join(k_sentences_in_order)


In [ ]:
def summarize_document_with_kmeans(document, k):
    #Embed sentences
    sentences, embeddings = embed_using_bert(model, tokenizer, document)

    if len(sentences) >= k:
        summary = cluster_sentence_embeddings(embeddings, sentences, k)
    else:

        summary = document if len(sentences) < 2 else ". ".join(sentences[:k])

    return summary

tqdm.pandas()  # Enable progress bar for DataFrame operations
df['cleaned'] = df['text'].apply(clean)
df['neural_summary'] = df['cleaned'].progress_apply(lambda text: summarize_document_with_kmeans(text, k=5))

100%|██████████| 20000/20000 [1:10:00<00:00,  4.76it/s]


In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 10)  # Adjust as needed to avoid truncation
df['neural_summary']

,neural_summary
0,Nursing progress Note 1 Resp recd pt on HIFI map 13 delta p 24. CBG at 1530 7. needs. Other PICC cultures pending. Efforts made to decrease interventionsstimulation
1,Continued ing. HR increased to 110s wdecrease in CVP LR bolus givencont to monitor assess fluid status. PLAN plan for Head CT this AM. cont q1hr neuro checks. cont to monitor neuro status ICP CPP EVD subdural drains drainage pupillary response UOP hemodynamicsassess need for fluid
2,wife and daughter expressed upset that they wer not present at time of extubation. positive bowel sounds. resp o2 sats on room air when awake 9799 . likes to splint with pillow. labs K3
3,Nicu Nursing Note 19000700 1Resp. A Altered resp. A Continues with spits with feeds. O Temp. 7 to 98
4,Sinus bradycardia premature ventricular contractions Possible old anterior infarct Lateralinferior STT changes are nonspecific
...,...
19995,Normal sinus rhythm rate 97 Left atrial enlargement Borderline low voltage in frontal leads Prior Inferior myocardial infarct Modest Nonspecific lateral T waves abnormalities Since last ECG no significant change Abnormal ECG
19996,Levophed was then increased to 0. Pt went to angio and was given Verapamil intrathecally for mod amt vasospasms. Neuro signs this pm Pt opens eyes to sternal rub and moves all extremities except right arm to deep pain. Resp. No active bowel sounds
19997,Neonatology Attending Note CPAP NP 5 21. RR60s. Mild rtxns. TF 150 cckday of BM32 w pr 115 IVF 35. ds 79
19998,Respiratory Therapy Pt extubated this AM to cool neb via face tent positive cuff leak but post extubation stridor noted. New ETT 7. 0 taped 22cmlip. See resp flowsheet for specific vent datachanges. Plan maintain support


In [ ]:
'''
   Reduce dataset to 10,000 cleaned text as we have limited GPU access time on collab
   From Reduced cleaned text docs we can still analyze summary quality effectively, while varying k.
'''
# Reduce the dataset to 10,000 documents for faster processing
df = df.sample(n=10000, random_state=42).reset_index(drop=True)

# Reapply the cleaned column processing if not already done
if 'cleaned' not in df.columns:
    df['cleaned'] = df['text'].apply(clean)



In [ ]:
#Changing k to 3

df['neural_summary'] = df['cleaned'].progress_apply(lambda text: summarize_document_with_kmeans(text, k=3))
df['neural_summary']

100%|██████████| 10000/10000 [35:19<00:00,  4.72it/s]


,neural_summary
0,Neo attending DOL 97 for this former 25 week infant. In NCO2 for treatment of CLD. PO PG
1,In clinic tachycardic to 130 s hypertensive. Pt reports no palpitations or racing HR as he did upon admission. Reports feeling better Plan Continue current medication regiment Consider changing propanolol to toprolol xl Chest xray APLAT to evaluate for CHF OOB and ambulate TID Encourage nutrition Continue to update pt and family on Hospital Transfer to floor when bed becomes available
2,TOTAL FLUIDS OF D10 WITH LYTES INCREASED TO 100 CCKG AT 0100. ABD BENIGN. 2
3,NP Note PE growing near term infant in open crib with sister. Name NI well perfused in RA. Name NI 158 sutures approximated eyes mildly edematous ng in place MMMP hest is symmetric with clear equal bs comfortbale CV RRR Last Name un 2515 murmur pulses2 Abd soft with active bs GU edematous labia EXT well developed MAE Neuroactive with good tone.
4,Attempts to sit up in the bed. No complaints of headache. HO dementia including Alzheimers Multi Infarct Assessment Pt very confused not answering questions correctly
...,...
9995,There is vicarious excretion of contrast into the gallbladder. The pancreas is unremarkable. IMPRESSION 1 Small amount of intraperitoneal low attenuation fluid slightly increased in amount since the prior study
9996,Please rule out PTX and pneumomediastinum and assess ETT position. An ET tube remains in place its tip just at the carina. 2
9997,5 Resp REVISIONS TO PATHWAY 5 Resp added Start date 21241221
9998,Sensation unable to test Toes upgoing bilaterally Pertinent Results HEAD CT 59 There are two large likely contiguous areas of intraparenchymal hemorrhage one centered at the left basal ganglia 215 measuring 5. 3 cm. Cerebral angiogram 59 no report at the time of this writing but the physician told the neurosurgery team that there was no evidence of AVManeurysm Brief Hospital Course The patient was evaluated in the ER by neurology and neurosurgery was called when she starting developing anisocoria


In [ ]:
#changing k to 7
df['neural_summary'] = df['cleaned'].progress_apply(lambda text: summarize_document_with_kmeans(text, k=7))
df['neural_summary']

100%|██████████| 10000/10000 [35:23<00:00,  4.71it/s]


,neural_summary
0,In NCO2 for treatment of CLD. In 2550 ccs of 100 O2. Remains on diurilKCL. Soft PFO m noted. PO PG. Improving po feeds RRR intermittent m CLear BS tachypneic Soft abdomen BS 2 pulses AP Preterm infant with CLD. Continue current plan for today.
1,33 yo adm from Hospital Clinic via ED with 4 mo ho wt loss tremor weakness anxiety. In clinic tachycardic to 130 s hypertensive. Transferred to ED where given propranolol and methimazole. NPN 07001100 Hypertension benign Assessment Pt presents with tachycardia and hypertension in setting of severe hyperthyroidism. SBP 120150 s goal 160. Pt reports no palpitations or racing HR as he did upon admission. Reports feeling better Plan Continue current medication regiment Consider changing propanolol to toprolol xl Chest xray APLAT to evaluate for CHF OOB and ambulate TID Encourage nutrition Continue to update pt and family on Hospital Transfer to floor when bed becomes available
2,NURSING PROGRESS NOTE 1. TOTAL FLUIDS OF D10 WITH LYTES INCREASED TO 100 CCKG AT 0100. ONE TOUCH 7589. ABD BENIGN. 2. LOVES PACIFIER HUNGRY. PARENTING MOM AND DAD IN ASKING APPROPRIATE QUESTIONS.
3,NP Note PE growing near term infant in open crib with sister. Name NI well perfused in RA. Name NI 158 sutures approximated eyes mildly edematous ng in place MMMP hest is symmetric with clear equal bs comfortbale CV RRR Last Name un 2515 murmur pulses2 Abd soft with active bs GU edematous labia EXT well developed MAE Neuroactive with good tone.
4,Complaining of headache and right hip pain. . Attempts to sit up in the bed. head ct today. Response Stable neuro status. HO dementia including Alzheimers Multi Infarct Assessment Pt very confused not answering questions correctly. Plan Monitor closely
...,...
9995,21311223 1212 AM CT ABDOMEN WCONTRAST CT PELVIS WCONTRAST Clip Clip Number Radiology 31888 Reason prior CT abd showing soft tissue injury and ascites please Admitting Diagnosis SAHPEDISTRIAN STRUCK Hospital 2 MEDICAL CONDITION 30 year old man sp trauma with rising lactate SAH and unable to do a clinical abd exam because pt somnolent REASON FOR THIS EXAMINATION prior CT abd showing soft tissue injury and ascites please eval further w contrast No contraindications for IV contrast FINAL REPORT INDICATION Status post trauma with rising lactate possible soft tissue injury and ascites. The chest tube appears malpositioned and possibly kinked. The pancreas is unremarkable. Patchy oral contrast is also seen within the small bowel and within the ascending colon. There is a small amount of ascites which has slightly increased in amount in the interval. PELVIS CT WITH IV CONTRAST The distal ureters bladder rectum and pelvic loops of bowel are unremarkable. Due to continued subcutaneous edema and low attenuation character of the fluid the fluid may be due to a third spacing process
9996,215553 709 AM CHEST PORTABLE AP Clip Clip Number Radiology 27240 Reason 58 year old man sp total thyroidectomy partial tracheal re Admitting Diagnosis THYROID CANCERSDA Hospital 4 MEDICAL CONDITION 58 year old man sp total thyroidectomy partial tracheal resection mediastinal lymph node dissection. Please rule out PTX and pneumomediastinum and assess ETT position. An ET tube remains in place its tip just at the carina. No CHF or frank airspace infiltrate is seen. IMPRESSION 1. This was communicated to Dr. 2
9997,5 Resp REVISIONS TO PATHWAY 5 Resp added Start date 21241221
9998,Admission Date 217059 Discharge Date 2170511 Date of Birth 212125 Sex F Service NEUROSURGERY Allergies Patient recorded as having No Known Allergies to Drugs AttendingFirst Name3 LF 1835 Chief Complaint headache hemiplegia Major Surgical or Invasive Procedure angiogram 217059 History of Present Illness 49 year old female ho MS First Name Titles Last Name Titles was at her PCPs office today when she had sudden onset of headache and right hemiplegia. 4 x 3. No areas of territorial infarction identified. Cerebral angiogram 59 no report at the time of this writi

In [ ]:
#Changing token embeddings to max
def embed_using_bert(bert_model, tokenizer, document):
    sentences = document.split(". ")
    sentence_embeddings = []

    for sentence_index, sentence in enumerate(sentences):
        # Tokenize the sentence and move input tensors to GPU if available
        tokens = tokenizer(sentence, return_tensors="pt", truncation=True, max_length=512).to(device)

        # Extract embeddings for tokens
        with torch.no_grad():
            outputs = bert_model(**tokens)

        # Get the token embeddings (hidden states of the last layer)
        token_embeddings = outputs.last_hidden_state.squeeze(0)

        # Compute the embedding for the sentence
        sentence_embedding = token_embeddings.max(dim=0).values.cpu().numpy()  # Move to CPU before converting to NumPy
        sentence_embeddings.append(sentence_embedding)

    return sentences, np.array(sentence_embeddings)

df['neural_summary'] = df['cleaned'].progress_apply(lambda text: summarize_document_with_kmeans(text, k=5))

df['neural_summary']

100%|██████████| 10000/10000 [35:59<00:00,  4.63it/s]


,neural_summary
0,Remains on diurilKCL. Soft PFO m noted. PO PG. Improving po feeds RRR intermittent m CLear BS tachypneic Soft abdomen BS 2 pulses AP Preterm infant with CLD. Continue current plan for today.
1,33 yo adm from Hospital Clinic via ED with 4 mo ho wt loss tremor weakness anxiety. FH strongly pos for thyroid dis likely Doctor Last Name 3479 Disease. To MICU 6 for close cardiac monitoring in setting of tachycardia hypertension and hyperthyroid. Action Propanolol increased to QID Methimazole Hospital1 Response HR 90100 s goal 80 s. SBP 120150 s goal 160
2,ABD BENIGN. 2. GD MAINTAINING TEMP ON OFF WARMER. LOVES PACIFIER HUNGRY. PARENTING MOM AND DAD IN ASKING APPROPRIATE QUESTIONS.
3,NP Note PE growing near term infant in open crib with sister. Name NI well perfused in RA. Name NI 158 sutures approximated eyes mildly edematous ng in place MMMP hest is symmetric with clear equal bs comfortbale CV RRR Last Name un 2515 murmur pulses2 Abd soft with active bs GU edematous labia EXT well developed MAE Neuroactive with good tone.
4,Attempts to sit up in the bed. head ct today. Response Stable neuro status. Plan To head ct today and monitor neuro status closely. Light on to orient patient
...,...
9995,There is a left basilar pneumothorax which appears slightly increase in size in the interval. There is vicarious excretion of contrast into the gallbladder. The pancreas is unremarkable. No free intraperitoneal contrast is present. An occult bowel injury is still not fully excluded and close clinical follow up is Over 21311223 1212 AM CT ABDOMEN WCONTRAST CT PELVIS WCONTRAST Clip Clip Number Radiology 31888 Reason prior CT abd showing soft tissue injury and ascites please Admitting Diagnosis SAHPEDISTRIAN STRUCK FINAL REPORT Cont recommended
9996,REASON FOR THIS EXAMINATION 58 year old man sp total thyroidectomy partial tracheal resection mediastinal lymph node dissection. PORTABLE CHEST Comparison is made to film from one day earlier. There is no pneumothorax. ET tube tip at carina this could be pulled back two centimeters for improved placement. Bibasilar atelectasis.
9997,5 Resp REVISIONS TO PATHWAY 5 Resp added Start date 21241221
9998,Neuro Mental status Not following commands no eye opening intubated sedated. 4 x 3. Significant mass effect caused by this enlarging intraparenchymal hemorrhage causing 10mm rightward subfalcine herniation and possible left uncal herniation. Her head CT at Hospital1 18 showed increased hemorrhage compared to the scan from the OSH. Dr


## BART

We will now use dedicated sequence-to-sequence neural networks. These models learn to map input sequences of text to different length output sequences of text. They do this with larged labeled summary datasets. It would be great to train (or even fine-tune) our own bio-medical summarization model, but unfortunately this requires datasets which do not exist (to my knowledge). Thus, we will use [BART](https://huggingface.co/facebook/bart-large), a general purpose encoder-decoder model from meta. This is the most opaque but potentially best performing summarization model, as it deals with actual text generation. Unlike the previous models, which pick and choose pre-existing texts, sequence-to-sequence models generate a completely new summary. Thus, there is only one step: model evaluation.

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

import torch
if torch.cuda.is_available():
    print('GPU available!')
    device = torch.cuda.current_device()
else:
    print('GPU unavailable - CPU will be used for all calculations')
    device = None

# Tokenizer and model loading for bart-large-cnn

tokenizer=BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model=BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn').to(device)


#for multi-line output, thanks to https://stackoverflow.com/questions/58890109/line-wrapping-in-collaboratory-google-results
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

GPU available!


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
# Transmitting the encoded inputs to the model.generate() function
inputs = tokenizer.batch_encode_plus([df.cleaned.iloc[10]],return_tensors='pt').to(device)
summary_ids =  model.generate(inputs['input_ids'], num_beams=4, max_length=150, early_stopping=True)
# Decoding and printing the summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)
print("\n===\n")
print(df.cleaned.iloc[10])

Infant is weaning on the 976. He is day 611 of ampi and gent. Infants wt was up 50g but he was not weighed yesterday so this is for the last 2 days. Lytes are still pnd at this writing. Infant is under triple phototx w his eyes covered.

===

1. Resp O Received infant on the 976 vent at MawP 10 and Delta P 20. He has since weaned X2 and is now at a Delta P of 18. FiO2 has been 2329 mostly 23. He has been sxned X 2 for a small amt of white secretions from his ETT and sxned several times for a large amt of oral secretions. Ls were coarse. Infants RR is not decernible. No desats no bradys. A Weaning the 976 a bit. P Continue as per planNICU protocol. 2. CV O No murmur was audible. Infant is not on pressors. His bp M is in the 40s. HR 130140s. Color is pinkruddy. A Stable. P Monitor. 3. FN O Infant remains NPO on 180cckd TF consisting of TPN lipids and D6W and 12 NS heparin for the UAC. The rest of the fluids are running through the DUV. Infants wt was up 50g but he was not weighed yesterd

In [ ]:
'''reduced dataset to 2000 cleaned texts as the entire dataset was expected to take 5 hours, GPU would'nt ave lasted that long'''
df = df.sample(n=2000, random_state=42).reset_index(drop=True)
# Reapply the cleaned column processing if not already done
if 'cleaned' not in df.columns:
    df['cleaned'] = df['text'].apply(clean)


In [ ]:

# Function to summarize using BART
def bart_summarize(text):
    # Check if text is empty or None
    if not text or text.strip() == "":
        return ""  # Return empty summary for empty text

    inputs = tokenizer.batch_encode_plus(
        [text],
        return_tensors='pt',
        truncation=True,  # Truncate if sequence is too long
        max_length=1024, # Set a maximum sequence length
        padding="max_length" # Pad shorter sequences to the maximum length
    ).to(device)

    summary_ids = model.generate(
        inputs['input_ids'],
        num_beams=4,
        max_length=150,
        early_stopping=True
    )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Apply the BART summarization to the reduced dataset
df['bart_summary'] = df['cleaned'].progress_apply(bart_summarize)


100%|██████████| 2000/2000 [47:14<00:00,  1.42s/it]


In [ ]:
df['bart_summary']

,bart_summary
0,Neo attending DOL 97 for this former 25 week infant. In NCO2 for treatment of CLD. In 2550 ccs of 100 O2. Remains on diurilKCL. Soft PFO m noted. Weight is 2230 up 20 gms on TF of 130 cckgday.
1,33 yo adm from Hospital Clinic via ED with 4 mo ho wt loss tremor weakness anxiety. FH strongly pos for thyroid dis likely Doctor Last Name 3479 Disease. In clinic tachycardic to 130 s hypertensive. Transferred to ED where given propranolol and methimazole. To MICU 6 for close cardiac monitoring.
2,NURSING PROGRESS NOTE 1. FN TONIGHTS WEIGHT DOWN 110 GRAMS. 2. GD MAINTAINING TEMP ON OFF WARMER. 3. PARENTING MOM AND DAD IN ASKING APPROPRIATE QUESTIONS.
3,NP Note PE growing near term infant in open crib with sister. Name NI well perfused in RA. NI 158 sutures approximated eyes mildly edematous ng in place. MMMP hest is symmetric with clear equal bs comfortbale CV RRR Last Name un 2515 murmur pulses2 Abd soft with active bs GU edem atous labia EXT well developed MAE Neuroactive with good tone.
4,82 year old who lives in a nursing home. On 117 had an unwitnessed fall. Complaining of headache and right hip pain. Unable to recall falling as pt suffers from dementia. Head ct scan showed traumatic SDH. Transferred to the Hospital1 for further neurological evaluation.
...,...
1995,Laminectomy and Tumor Resection Assessment Pt incision site appears approximated clean intact with scant serosanguinous drainage feels crampingnumbnesstingling in lower legs Action MRI of lumbar region done today continue to assess neuro checks every 1 hour to remain flat manage his pain Name8 MD MD assessed for lower leg cramping Response Tolerated MRI procedure Pt aware of plan to remain on bedrest and flat on back until 21981217.
1996,NPN 71998 1. Infant conts on ad lib q3hr feed schedule. BME 20. Taking 3555cc each feeds. Ds prior to feeds 45 and 56 thus far. Abd soft wactive BS. No spits. VS normally. P Not ready for NBN this time.
1997,Multiple vent changes made for ABGs as documented on flow sheet. Neut x1 given this am. Last ABG at 1700 7. 2251. 642 increased for that gas. Current MAP 9 delta p 16. FIO2 2125. LS clear this am when position changed. ICSubC retractions. sxd x1 for sm secretions.
1998,1 Alt. in Resp. O Remains in RA with sats 96100. Breath sounds clear and . RR 30s60s with intermittent mild SC retractions. No apnea bradycardia or desats noted today. A Doing well in RA P Continue to monitor and observe. Document any spells.


In [ ]:
# Function to summarize with custom num_beams and max_length
def bart_summarize_with_params(text, num_beams=4, max_length=150):
    inputs = tokenizer.batch_encode_plus([text], return_tensors='pt', truncation=True).to(device)
    summary_ids = model.generate(
        inputs['input_ids'],
        num_beams=num_beams,
        max_length=max_length,
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Example: Apply to a single document with different parameters
text = df.cleaned.iloc[10]  # Example cleaned document

# Experiment with different values of num_beams and max_length
results = {
    "num_beams=2, max_length=100": bart_summarize_with_params(text, num_beams=2, max_length=100),
    "num_beams=4, max_length=150": bart_summarize_with_params(text, num_beams=4, max_length=150),
    "num_beams=4, max_length=200": bart_summarize_with_params(text, num_beams=4, max_length=200),
    "num_beams=6, max_length=200": bart_summarize_with_params(text, num_beams=6, max_length=200),
}

# Print results for analysis
for params, summary in results.items():
    print(f"\nParameters: {params}\nSummary: {summary}")



Parameters: num_beams=2, max_length=100
Summary: 71 year old man with acute onset R facearmleg weakness today L basal ganglia hemorrhage. Evaluate for underlying vascular malformation or mass. No contraindications for IV contrast. MRI of the BRAIN. A 33 x 13 mm left basal Ganglia hemorrhages with minimal surrounding vasogenic edema.

Parameters: num_beams=4, max_length=150
Summary: 71 year old man with acute onset R facearmleg weakness today L basal ganglia hemorrhage. eval for underlying vascular malformation or mass fu size of bleed. Please do study 621 am. No contraindications for IV contrast. MRI of the brain: A 33 x 13 mm left basal ganglia hemorrhage with minimal surrounding vasogenic edema is stable in size and location.

Parameters: num_beams=4, max_length=200
Summary: 71 year old man with acute onset R facearmleg weakness today L basal ganglia hemorrhage. eval for underlying vascular malformation or mass fu size of bleed. Please do study 621 am. No contraindications for IV co